## Solución MEF con Sympy

In [1]:
#importar librerías necesarias
import sympy as sp
sp.init_printing(use_unicode=True)

Variables simbólicas a utilizar

In [2]:
x, L, K, Q, q, a_1, a_2, a_3, T_L = sp.symbols('x, L, K, Q, q, a_1, a_2, a_3, T_L') 

**Usando la forma matricial, la temperatura se aproxima a la forma: $T(x)=a\phi$, declarar las marices $\phi = [\phi_i, \phi_{i+1}]$, $d\phi = [d\phi_i, d\phi_{i+1}]$, $a=[a_i, a_{i+1}]^T$:**

In [3]:
phi = sp.Matrix([1-2*x/L, 2*x/L]).T
phi

⎡    2⋅x  2⋅x⎤
⎢1 - ───  ───⎥
⎣     L    L ⎦

In [4]:
dphi = sp.diff(phi, x)
dphi

⎡-2   2⎤
⎢───  ─⎥
⎣ L   L⎦

In [5]:
# a para el primer elemento
a = sp.Matrix([a_1, a_2])
a

⎡a₁⎤
⎢  ⎥
⎣a₂⎦

Tomando la ecuación de la aproximación a nuestra ecuación diferencial en su forma matricial:

$$K \left[ \int_0^{L/2} \left[ \frac {d}{dx}\phi \right] ^T \left[ \frac {d}{dx}\phi \right] dx \right] a - Q \int_0^{L/2} \phi^T dx - \begin{bmatrix}q \\ 0 \end{bmatrix} = 0$$



In [6]:
eq1 = K*sp.integrate(dphi.T * dphi, (x,0,L/2))*a - Q*sp.integrate(phi.T, (x,0,L/2)) - sp.Matrix([q,0])
eq1

⎡2⋅K⋅a₁   2⋅K⋅a₂   L⋅Q    ⎤
⎢────── - ────── - ─── - q⎥
⎢  L        L       4     ⎥
⎢                         ⎥
⎢   2⋅K⋅a₁   2⋅K⋅a₂   L⋅Q ⎥
⎢ - ────── + ────── - ─── ⎥
⎣     L        L       4  ⎦

In [7]:
# renombrando a para el segundo elemento
a = sp.Matrix([a_2, a_3])
a

⎡a₂⎤
⎢  ⎥
⎣a₃⎦

In [8]:
eq2 =  K*sp.integrate(dphi.T * dphi, (x,0,L/2))*a - Q*sp.integrate(phi.T, (x,0,L/2)) - sp.Matrix([0,0])
eq2

⎡ 2⋅K⋅a₂   2⋅K⋅a₃   L⋅Q ⎤
⎢ ────── - ────── - ─── ⎥
⎢   L        L       4  ⎥
⎢                       ⎥
⎢  2⋅K⋅a₂   2⋅K⋅a₃   L⋅Q⎥
⎢- ────── + ────── - ───⎥
⎣    L        L       4 ⎦

In [10]:
# inicializando en zeros
eq = sp.Matrix([0,0,0])
eq

⎡0⎤
⎢ ⎥
⎢0⎥
⎢ ⎥
⎣0⎦

In [11]:
#Ensamblando las matrices obtenidas en eq1 y eq2
eq[0] = eq1[0]
eq[1] = eq1[1] + eq2[0]
eq[2] = eq2[1]
eq

⎡   2⋅K⋅a₁   2⋅K⋅a₂   L⋅Q        ⎤
⎢   ────── - ────── - ─── - q    ⎥
⎢     L        L       4         ⎥
⎢                                ⎥
⎢  2⋅K⋅a₁   4⋅K⋅a₂   2⋅K⋅a₃   L⋅Q⎥
⎢- ────── + ────── - ────── - ───⎥
⎢    L        L        L       2 ⎥
⎢                                ⎥
⎢      2⋅K⋅a₂   2⋅K⋅a₃   L⋅Q     ⎥
⎢    - ────── + ────── - ───     ⎥
⎣        L        L       4      ⎦

In [12]:
# tomamos las dos primeras ecuaciones por que ya conocemos que a3=TL
eq_incognitas = sp.Matrix(eq[0:2])
eq_incognitas

⎡   2⋅K⋅a₁   2⋅K⋅a₂   L⋅Q        ⎤
⎢   ────── - ────── - ─── - q    ⎥
⎢     L        L       4         ⎥
⎢                                ⎥
⎢  2⋅K⋅a₁   4⋅K⋅a₂   2⋅K⋅a₃   L⋅Q⎥
⎢- ────── + ────── - ────── - ───⎥
⎣    L        L        L       2 ⎦

In [13]:
sol = sp.solve(eq_incognitas.subs({'a_3':'T_L'}), a_1, a_2) #Solucionando el sistema de ecuaciones conociendo a_3=TL
sol

Estas deberán ser subsituidas en la ecuación (aproximada) de $T(x)$:

$$T(x) \approx a_1 \phi_1 (x) + a_2 \phi_2 (x) + a_3 \phi_3 (x) $$

$$T(x) \approx \left(\frac {qL} {K} + \frac {QL^2} {2K} + T_L \right) \phi_1 (x) + \left( \frac {qL} {2K} + \frac {3QL^2} {8K} + T_L \right) \phi_2 (x) + T_L \phi_3 (x) $$